<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%205/homework_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pymagnitude

     |████████████████████████████████| 5.4MB 4.7MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918206 sha256=e420776885416eb8757c92a71d07898c703d97435ae63352a4d0f484669b8a88
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude


In [0]:
from pymagnitude import *

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# to view contents, run following line
# !ls drive/My\ Drive/CIS-530/Homework\ 5/Data

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
texts_path = 'drive/My\ Drive/CIS-530/Homework\ 5/Data/data'
vectors_path = 'drive/My\ Drive/CIS-530/Homework\ 5/Data/vectors'